In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from pathlib import Path

In [3]:
class HyperLoss(tf.keras.losses.Loss):
    def __init__(self , threshold=5,**kwargs):
        self.threshold = threshold
        super(HyperLoss ,self).__init__(**kwargs)
    def call(self, y_true ,  y_pred):
        error = y_true - y_pred
        is_samll = tf.abs(error)<self.threshold
        square_loss = tf.square(error)/2.0
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2.0
        return tf.where(is_samll , square_loss , linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold":self.threshold}


In [5]:
model = tf.keras.models.load_model("modelv2.h5" , custom_objects={"HyperLoss":HyperLoss()})

In [7]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 1)]      0           []                               
                                                                                                  
 conv1d_1 (Conv1D)              (None, 64, 16)       64          ['input_1[0][0]']                
                                                                                                  
 conv1d_2 (Conv1D)              (None, 64, 16)       784         ['conv1d_1[0][0]']               
                                                                                                  
 re_lu (ReLU)                   (None, 64, 16)       0           ['conv1d_2[0][0]']               
                                                                                              

In [8]:
converter =tf.lite.TFLiteConverter.from_keras_model(model)

In [10]:
converter.optimizations =[tf.lite.Optimize.DEFAULT]

In [12]:
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

In [13]:
with open("model_tflite.tflite","wb") as f:
    f.write(converter.convert())

INFO:tensorflow:Assets written to: C:\Users\ADMINI~1\AppData\Local\Temp\tmpkio06qld\assets


INFO:tensorflow:Assets written to: C:\Users\ADMINI~1\AppData\Local\Temp\tmpkio06qld\assets


In [4]:
import tensorflow as tf
interpreter = tf.lite.Interpreter("model_tflite.tflite")
interpreter.allocate_tensors()

In [5]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [6]:
input_details

[{'name': 'serving_default_input_1:0',
  'index': 0,
  'shape': array([ 1, 64,  1]),
  'shape_signature': array([-1, 64,  1]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [7]:
output_details

[{'name': 'StatefulPartitionedCall:0',
  'index': 351,
  'shape': array([1, 1]),
  'shape_signature': array([-1,  1]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'StatefulPartitionedCall:1',
  'index': 352,
  'shape': array([1, 1]),
  'shape_signature': array([-1,  1]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [8]:
import numpy as np
example = np.random.random((1,64))
example

array([[0.44395927, 0.66781414, 0.06677912, 0.23473246, 0.94821627,
        0.07079948, 0.31435782, 0.54415069, 0.59503533, 0.45607879,
        0.48688049, 0.49444145, 0.39159317, 0.15479789, 0.45664297,
        0.03933922, 0.18458209, 0.7258324 , 0.89928054, 0.54675961,
        0.45297707, 0.88660501, 0.64245159, 0.20891946, 0.52637594,
        0.03640396, 0.56467148, 0.57538171, 0.83667835, 0.62281711,
        0.52986397, 0.94190843, 0.53511526, 0.09999469, 0.46623984,
        0.2323737 , 0.54268983, 0.90524892, 0.15905245, 0.47562398,
        0.31032806, 0.88084353, 0.42609383, 0.46402744, 0.36876996,
        0.81902908, 0.29798531, 0.13097442, 0.93377438, 0.5313319 ,
        0.62631491, 0.94963611, 0.99428832, 0.11696301, 0.31879157,
        0.65374289, 0.75354945, 0.80623153, 0.61488753, 0.33391801,
        0.67162463, 0.37225407, 0.92900314, 0.43794217]])

In [9]:
example = example[...,np.newaxis]

In [10]:
input_details[0]

{'name': 'serving_default_input_1:0',
 'index': 0,
 'shape': array([ 1, 64,  1]),
 'shape_signature': array([-1, 64,  1]),
 'dtype': numpy.float32,
 'quantization': (0.0, 0),
 'quantization_parameters': {'scales': array([], dtype=float32),
  'zero_points': array([], dtype=int32),
  'quantized_dimension': 0},
 'sparsity_parameters': {}}

In [11]:
interpreter.set_tensor(input_details[0]['index'], example.astype(np.float32))

In [12]:
interpreter.allocate_tensors()

In [13]:
tensor_index = interpreter.get_input_details()[0]['index']
input_tensor_z= tf.convert_to_tensor(example, np.float32)
interpreter.set_tensor(tensor_index, input_tensor_z)

In [43]:
max = interpreter.get_tensor(output_details[0]['index'])
min = interpreter.get_tensor(output_details[1]['index'])


In [44]:
min ,max

(array([[64.65507]], dtype=float32), array([[127.89897]], dtype=float32))

In [14]:
interpreter.invoke()

In [15]:
output_details

[{'name': 'StatefulPartitionedCall:0',
  'index': 351,
  'shape': array([1, 1]),
  'shape_signature': array([-1,  1]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'StatefulPartitionedCall:1',
  'index': 352,
  'shape': array([1, 1]),
  'shape_signature': array([-1,  1]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [16]:
max = interpreter.get_tensor(output_details[0]['index'])
min = interpreter.get_tensor(output_details[1]['index'])


In [17]:
print("min value is ", min , "max value is ", max)

min value is  [[73.12287]] max value is  [[129.95854]]


In [18]:
example.shape

(1, 64, 1)

In [16]:
max , min

(array([[138.27809]], dtype=float32), array([[76.20431]], dtype=float32))

In [48]:
tf.__version__

'2.11.0'

In [49]:
np.__version__

'1.21.6'

In [50]:
pd.options.display.max_columns =None
pd.options.display.max_rows = None

In [28]:
#scale data between -1 and 1
min = 0
max = 300

In [56]:
min = 0
max = 300
def normalize(list_data):
    data = np.array(list_data)
    data = (2*(data - min)/(max -min )) - 1
    data = data[np.newaxis,:,np.newaxis]
    return data

In [19]:
300 / 64

4.6875

In [21]:
64*4

256

In [57]:
data_items = [i for i in range(0, 256,4)]

In [58]:
len(data_items)

64

In [59]:
data = normalize(data_items)

In [60]:
data.min()

-1.0

In [61]:
data.max()

0.6799999999999999

In [40]:
data.shape

(1, 64, 1)